<a href="https://colab.research.google.com/github/Carol-Coutinho/Funcoes-uteis-para-engenharia/blob/main/Fator_de_Compressibilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Calculadora de Fator de Compressibilidade  (Z)

### Caminho 1: Usando banco de dados prórpio.

Nesse caso o banco de dados está em unidade do S.I.

https://hackernoon.com/how-to-create-a-compressibility-factor-calculator-in-python-jr183306 

In [33]:
# Preparando kernel
import pandas as pd
#import pickle
!pip3 install pickle5
import pickle5 as pickl
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
# Para ler/carregar
with open('/content/databank_properties.pickle', 'rb') as handle:
    dados, all_units = pickl.load(handle)

FileNotFoundError: ignored

In [ ]:
dados.columns

In [ ]:
# Buscando o primeiro componente:
filtro1 = dados.name.str.contains('METHANE')
dados[filtro1]

Metano = 295

In [ ]:
dados_met = dados.iloc[295]
dados_met

## Caminho 2: Sem banco de dados

Baseado no código disponível em:

 https://hackernoon.com/how-to-create-a-compressibility-factor-calculator-in-python-jr183306 

Cálculo das pressões e temperaturas reduzidas:

In [21]:
# Reduced temperature
# T and Tc have the units of °R.
def reduced_temperature(temperature, tc):
    tr = temperature / tc
    return tr

In [22]:
# Reduced pressure
# P and Pc have the units of psia
def reduced_pressure(pressure, pc):
    pr = pressure / pc
    return pr

Cálculo das constantes de Redlich-Kwong

In [23]:
# Redlich-Kwong constant A
def redlich_kwong_constant_a(pr, tr):
    a = 0.42748 * (pr / tr ** 2.5)
    return a


# Redlich-Kwong constant B
def redlich_kwong_constant_b(pr, tr):
    b = 0.08664 * (pr / tr)
    return b

Cálculo das constantes cúbicas:

In [24]:
# Cubic constant α
def cubic_constant_alpha(a, b):
    alpha = (1 / 3) * (3 * (a - b - b ** 2) - 1)
    return alpha


# Cubic constant β
def cubic_constant_beta(a, b):
    beta = (1 / 27) * (-2 + (9 * (a - b - b ** 2)) - (27 * a * b))
    return beta

Cálculo de D:

In [25]:
# Discriminant
def discriminant(alpha, beta):
    d = (beta ** 2 / 4) + (alpha ** 3 / 27)
    return d

Cálculo das constantes A, B e $Θ$:

In [26]:
# Solution Constant A*
def solution_constant_a_star(beta, d):
    a_star = np.cbrt((-beta / 2) + np.sqrt(d))
    return a_star


# Solution Constant B*
def solution_constant_b_star(beta, d):
    b_star = np.cbrt((-beta / 2) - np.sqrt(d))
    return b_star


# Solution Constant Theta
def solution_constant_theta(beta, alpha):
    theta = math.acos(-sign(beta) * (math.sqrt((beta ** 2 / 4) / (-alpha ** 3 / 27))))
    return theta

Cálculo de $β$

In [27]:
# SIGN(β)
# If β > 0, SIGN(β) = 1
# If β = 0, SIGN(β) = 0
# If β < 0, SIGN(β) = -1
def sign(beta):
    if beta > 0:
        sign_beta = 1
    elif beta < 0:
        sign_beta = -1
    else:
        sign_beta = 0
    return sign_beta

Cálculo das raízes de Z:

In [28]:
# Trial Root Z1
def trial_root_z1(a_star, b_star):
    z1 = a_star + b_star + 1 / 3
    return z1


# Trial Root Zi
# for i = 2, 3
def trial_root_zi(a_star, b_star, i):
    zi = (-(1 / 2) * (a_star + b_star)) + ((1 / 3) * i)
    return zi


# Trial Root Zi+1
# for i = 1, 2, 3
def trial_root_zi1(alpha, theta, i):
    zi1 = (2 * math.sqrt(- alpha / 3) * math.cos((theta / 3) + (i * ((math.pi * 2) / 3)))) + (1 / 3)
    return zi1

Tabela das contantes crísticas para alguns gases:

In [29]:
def table_gas_critical_constants(gas_name):
    """
    Retrieves gas critical constants table data
    :param gas_name: Gas name
    :type gas_name: str
    :return: Gas critical constants data
    :rtype: list
    """
    gas_critical_constants = {
        "Air": [238.56, 549.11, 28.965],
        "Carbon Dioxide": [547.60, 1070.600, 44.011],
        "Hydrogen": [59.82, 190.82, 2.02],
        "Methane": [343.90, 673.100, 16.04],
        "Nitrogen": [227.25, 492.420, 28.13],
        "Propane": [666.00, 618.700, 44.09],
        "Typical Natural Gas": [360.00, 777.373, 17.185]
    }
    return gas_critical_constants.get(gas_name)

Definição da função de compressibilidade (Z): 

In [35]:
def compressibility_factor_calc(gas, temperature, pressure):
    """
    Calculates compressibility factor
    :param gas: Gas name
    :type gas: str
    :param temperature: Temperature
    :type temperature: float
    :param pressure: Pressure
    :type pressure: float
    :return: Compressibility factor(z)
    :rtype: float
    """
    values = table_gas_critical_constants(gas)
    tc = values[0]
    pc = values[1]
    tr = reduced_temperature(temperature, tc)
    pr = reduced_pressure(pressure, pc)
    a = redlich_kwong_constant_a(pr, tr)
    b = redlich_kwong_constant_b(pr, tr)
    alpha = cubic_constant_alpha(a, b)
    beta = cubic_constant_beta(a, b)
    d = discriminant(alpha, beta)
    if d < 0:
        theta = solution_constant_theta(beta, alpha)
        z1 = trial_root_zi1(alpha, theta, 1)
        z2 = trial_root_zi1(alpha, theta, 2)
        z3 = trial_root_zi1(alpha, theta, 3)
    else:
        a_star = solution_constant_a_star(beta, d)
        b_star = solution_constant_b_star(beta, d)
        if d == 0:
            z1 = trial_root_z1(a_star, b_star)
            z2 = trial_root_zi(a_star, b_star, 2)
            z3 = trial_root_zi(a_star, b_star, 3)
        else:
            z = trial_root_z1(a_star, b_star)
            return z
    z = max(z1, z2, z3)
    return z

Teste da função:

In [36]:
# Dados:

P = 700 #pisa
Temp = 68 + 459.67 #ºF

In [38]:
Z_met = compressibility_factor_calc("Methane", Temp, P)
Z_met

0.9117596444632956